In [168]:
from scenariogeneration import xosc

In [169]:
# Fixed parameters
MONDEO_ID = "1"
CAMERA_DRONE_ID = "5"
MONDEO_CATALOG_NAME = "mondeo"
DRONE_CATALOG_NAME = "camera_drone"

CATALOG_PATH = "../Catalogs/Vehicles"
CATALOG_NAME = "VehicleCatalog"
CONTROLLER_CATALOG_PATH = "../Catalogs/Controllers"
CONTROLLER_CATALOG_NAME = "ControllerCatalog"

OPENDRIVE_PATH = "../odr/GaragePlan.xodr"
SCENARIO_NAME = "GaragePlanScenario"
SCENARIO_FILE_NAME = SCENARIO_NAME + ".xosc"

In [170]:
# Dynamic parameters
camera_drone_speed = 30 / 3.6 # m/s
camera_drone_acceleration = 1.76 # m/s^2
camera_drone_retardation = 1.96 # m/s^2
camera_drone_O5_point = xosc.LanePosition(0, 2.5, 1, 2)

# Dynamic parameters
mondeo_speed = 15 / 3.6 # m/s
mondeo_slow_speed = 10 / 3.6 # m/s
mondeo_acceleration = 0.51 # m/s^2
mondeo_retardation = 2.17 # m/s^2
mondeo_O1_point = xosc.LanePosition(8, 0, 1, 3)
mondeo_trigger_positions = {
	CAMERA_DRONE_ID: xosc.LanePosition(0, 0, 1, 3)
}
mondeo_slow_down_position = xosc.LanePosition(2, 0, 1, 3)
mondeo_accelerate_position = xosc.LanePosition(1.0940648317337036, 0, 1, 12)
mondeo_brake_position = xosc.LanePosition(12, 0, 1, 0)

denm_trigger_speed = 10 / 3.6 # m/s

In [171]:
# Preamble
init = xosc.Init()
osc_params = xosc.ParameterDeclarations()
osc_catalogs = xosc.Catalog()
osc_catalogs.add_catalog(CATALOG_NAME, CATALOG_PATH)
osc_catalogs.add_catalog(CONTROLLER_CATALOG_NAME, CONTROLLER_CATALOG_PATH)
osc_road_network = xosc.RoadNetwork(roadfile=OPENDRIVE_PATH)
osc_entities = xosc.Entities()
osc_storyboard = xosc.StoryBoard(init)
osc_act = xosc.Act("start")

camera_drone_maneuver_group = xosc.ManeuverGroup(CAMERA_DRONE_ID + ",maneuver_group")
camera_drone_maneuver = xosc.Maneuver(CAMERA_DRONE_ID + ",maneuver")
camera_drone_maneuver_group.add_actor(CAMERA_DRONE_ID)

mondeo_maneuver_group = xosc.ManeuverGroup(MONDEO_ID + ",maneuver_group")
mondeo_maneuver = xosc.Maneuver(MONDEO_ID + ",maneuver")
mondeo_maneuver_group.add_actor(MONDEO_ID)

In [172]:
# Starting position
# Starting positions
osc_entities.add_scenario_object(MONDEO_ID,
	xosc.CatalogReference(CATALOG_NAME, MONDEO_CATALOG_NAME),
	xosc.CatalogReference(CONTROLLER_CATALOG_NAME, "externalController"))
osc_entities = osc_entities.add_scenario_object(
	CAMERA_DRONE_ID,
	xosc.CatalogReference(CATALOG_NAME, DRONE_CATALOG_NAME),
	xosc.CatalogReference(CONTROLLER_CATALOG_NAME, "externalController"))
drone_start_teleport = xosc.TeleportAction(camera_drone_O5_point)
drone_set_speed = xosc.AbsoluteSpeedAction(
	camera_drone_speed,
	xosc.TransitionDynamics(
		xosc.DynamicsShapes.linear,
		xosc.DynamicsDimension.rate,
		camera_drone_acceleration
	)
)

mondeo_start_teleport = xosc.TeleportAction(mondeo_O1_point)
mondeo_set_speed = xosc.AbsoluteSpeedAction(
	mondeo_speed,
	xosc.TransitionDynamics(
		xosc.DynamicsShapes.linear,
		xosc.DynamicsDimension.rate,
		mondeo_acceleration
	),
)

init.add_init_action(CAMERA_DRONE_ID, drone_start_teleport)
init.add_init_action(CAMERA_DRONE_ID, drone_set_speed)
init.add_init_action(MONDEO_ID, mondeo_start_teleport)
init.add_init_action(MONDEO_ID, mondeo_set_speed)

In [173]:
# Camera drone movement profile:
# accelerate to top speed,
# follow a trajectory,
# then decelerate to a stop
times = []
positions = [
	xosc.LanePosition(0, 2.5, 1, 2),
	xosc.LanePosition(14.9, 2.5, 1, 2),
	xosc.LanePosition(0, 2.5, 1, 3),
	xosc.LanePosition(10, 2.5, 1, 3)
]
polyline = xosc.Polyline(times, positions)
camera_drone_trajectory = xosc.Trajectory(CAMERA_DRONE_ID + ",start_follow_trajectory", closed=False)
camera_drone_trajectory.add_shape(polyline)
camera_drone_follow_trajectory = xosc.FollowTrajectoryAction(
	camera_drone_trajectory,
	xosc.FollowingMode.position,
	xosc.ReferenceContext.relative,
	1,
	0
)
camera_drone_set_speed = xosc.AbsoluteSpeedAction(
	camera_drone_speed,
	xosc.TransitionDynamics(
		xosc.DynamicsShapes.linear,
		xosc.DynamicsDimension.rate,
		camera_drone_acceleration
	),
)
camera_drone_brake = xosc.AbsoluteSpeedAction(
	0,
	xosc.TransitionDynamics(
		xosc.DynamicsShapes.linear,
		xosc.DynamicsDimension.rate,
		-camera_drone_retardation
	),
)

# Stop
drone_brake = xosc.AbsoluteSpeedAction(
	0,
	xosc.TransitionDynamics(
		xosc.DynamicsShapes.linear,
		xosc.DynamicsDimension.time,
		-camera_drone_acceleration
	)
)
drone_reach_brake_position = xosc.EntityTrigger(
    name=CAMERA_DRONE_ID + ",reach_brake_position",
    delay=0,
    conditionedge=xosc.ConditionEdge.rising,
    entitycondition=xosc.ReachPositionCondition(positions[-1], 1.0), # tolerance
    triggerentity=CAMERA_DRONE_ID
)
drone_brake_event = xosc.Event(
	CAMERA_DRONE_ID + ",stop_event",
    xosc.Priority.overwrite,
)

drone_brake_event.add_trigger(drone_reach_brake_position)
drone_brake_event.add_action(CAMERA_DRONE_ID + ",brake_to_stop", drone_brake)
camera_drone_maneuver.add_event(drone_brake_event)

In [174]:
# Mondeo movement profile:
# accelerate to top speed,
# slow down before turn,
# turn,
# accelerate to top speed,

# Start moving
times = []
positions = [
    mondeo_O1_point,
    xosc.LanePosition(0, 0, 1, 3),
    xosc.LanePosition(8.0940648317337036, 0, 1, 12),
    xosc.LanePosition(7.0940648317337036, 0, 1, 12),
    xosc.LanePosition(6.0940648317337036, 0, 1, 12),
    xosc.LanePosition(5.0940648317337036, 0, 1, 12),
    xosc.LanePosition(4.0940648317337036, 0, 1, 12),
    xosc.LanePosition(3.0940648317337036, 0, 1, 12),
    xosc.LanePosition(2.0940648317337036, 0, 1, 12),
    xosc.LanePosition(1.0940648317337036, 0, 1, 12),
    xosc.LanePosition(33, 0, 1, 0),
    xosc.LanePosition(20, 0, 1, 0),
    mondeo_brake_position
]
polyline = xosc.Polyline(times, positions)
trajectory = xosc.Trajectory(
    MONDEO_ID + ",init_follow_trajectory", closed=False)
trajectory.add_shape(polyline)
mondeo_follow_trajectory = xosc.FollowTrajectoryAction(
    trajectory,
    xosc.FollowingMode.position,
    xosc.ReferenceContext.relative,
    1,
    0
)
mondeo_scenario_started = xosc.ValueTrigger(
    MONDEO_ID + ",scenario_started",
    0,
    xosc.ConditionEdge.none,
    xosc.SimulationTimeCondition(0, xosc.Rule.greaterThan)
)
mondeo_follow_trajectory_event = xosc.Event(
    MONDEO_ID + ",mondeo_follow_trajectory_event",
    xosc.Priority.parallel
)
mondeo_follow_trajectory_event.add_trigger(mondeo_scenario_started)
mondeo_follow_trajectory_event.add_action(
    MONDEO_ID + ",init_follow_trajectory", mondeo_follow_trajectory)
mondeo_maneuver.add_event(mondeo_follow_trajectory_event)

# Slow down
mondeo_slow_down = xosc.AbsoluteSpeedAction(
	mondeo_slow_speed,
	xosc.TransitionDynamics(
		xosc.DynamicsShapes.linear,
		xosc.DynamicsDimension.rate,
		-mondeo_retardation
	),
)
mondeo_reach_slow_down_position = xosc.EntityTrigger(
	MONDEO_ID + ",reach_slow_down_position",
	delay=0,
	conditionedge=xosc.ConditionEdge.none,
	entitycondition=xosc.ReachPositionCondition(mondeo_slow_down_position, 1),
	triggerentity=MONDEO_ID
)
mondeo_slow_down_event = xosc.Event(
	MONDEO_ID + ",mondeo_slow_down_event",
	xosc.Priority.parallel
)
mondeo_slow_down_event.add_trigger(mondeo_reach_slow_down_position)
mondeo_slow_down_event.add_action(MONDEO_ID + ",slow_down", mondeo_slow_down)
mondeo_maneuver.add_event(mondeo_slow_down_event)

# Accelerate
mondeo_accelerate = xosc.AbsoluteSpeedAction(
	mondeo_speed,
	xosc.TransitionDynamics(
		xosc.DynamicsShapes.linear,
		xosc.DynamicsDimension.rate,
		mondeo_acceleration
	),
)
mondeo_reach_accelerate_position = xosc.EntityTrigger(
	MONDEO_ID + ",reach_accelerate_position",
	delay=0,
	conditionedge=xosc.ConditionEdge.none,
	entitycondition=xosc.ReachPositionCondition(mondeo_accelerate_position, 1),
	triggerentity=MONDEO_ID
)
mondeo_accelerate_event = xosc.Event(
	MONDEO_ID + ",mondeo_accelerate_event",
	xosc.Priority.parallel
)
mondeo_accelerate_event.add_trigger(mondeo_reach_accelerate_position)
mondeo_accelerate_event.add_action(MONDEO_ID + ",mondeo_accelerate", mondeo_accelerate)
mondeo_maneuver.add_event(mondeo_accelerate_event)

# Brake
mondeo_brake = xosc.AbsoluteSpeedAction(
    0,
    xosc.TransitionDynamics(
        xosc.DynamicsShapes.linear,
        xosc.DynamicsDimension.rate,
        -mondeo_retardation
    ),
)
mondeo_reach_brake_position = xosc.EntityTrigger(
    MONDEO_ID + ",reach_brake_position",
    delay=0,
    conditionedge=xosc.ConditionEdge.none,
    entitycondition=xosc.ReachPositionCondition(mondeo_brake_position, 1),
    triggerentity=MONDEO_ID
)
mondeo_brake_event = xosc.Event(
    MONDEO_ID + ",mondeo_brake_event",
    xosc.Priority.parallel
)
mondeo_brake_event.add_trigger(mondeo_reach_brake_position)
mondeo_brake_event.add_action(MONDEO_ID + ",brake_to_stop", mondeo_brake)
mondeo_maneuver.add_event(mondeo_brake_event)


In [175]:
# Mondeo triggers camera drone
mondeo_reached_camera_drone_trigger_position = xosc.EntityTrigger(
	name=MONDEO_ID + ",reached_camera_drone_trigger_position",
	delay=0,
	conditionedge=xosc.ConditionEdge.none,
	entitycondition=xosc.ReachPositionCondition(mondeo_trigger_positions[CAMERA_DRONE_ID], 2),
	triggerentity=MONDEO_ID
)

camera_drone_move_event = xosc.Event(
	CAMERA_DRONE_ID + ",start_move_event",
	xosc.Priority.parallel
)
camera_drone_move_event.add_trigger(mondeo_reached_camera_drone_trigger_position)
camera_drone_move_event.add_action(CAMERA_DRONE_ID + ",start_follow_trajectory", camera_drone_follow_trajectory)
camera_drone_move_event.add_action(CAMERA_DRONE_ID + ",set_speed", camera_drone_set_speed)
camera_drone_maneuver.add_event(camera_drone_move_event)

In [176]:
# Mondeo object triggers DENM warning

### DENM Action ###
mondeo_high_speed_detected = xosc.EntityTrigger(
	name=MONDEO_ID + ",high_speed_detected",
	delay=0,
	conditionedge=xosc.ConditionEdge.none,
	entitycondition=xosc.SpeedCondition(denm_trigger_speed, xosc.Rule.greaterThan),
	triggerentity=MONDEO_ID)

mondeo_denm_action = xosc.VisibilityAction(graphics=True, traffic=True, sensors=True)

denm_osc_event = xosc.Event(
	MONDEO_ID + ",high_speed_event",
	xosc.Priority.parallel,
)

denm_osc_event.add_trigger(mondeo_high_speed_detected)
denm_osc_event.add_action(MONDEO_ID + ",send_denm",
							mondeo_denm_action)
mondeo_maneuver.add_event(denm_osc_event)

In [177]:
# Collect into a scenario and write to file
mondeo_maneuver_group.add_maneuver(mondeo_maneuver)
camera_drone_maneuver_group.add_maneuver(camera_drone_maneuver)
osc_act.add_maneuver_group(mondeo_maneuver_group)
osc_act.add_maneuver_group(camera_drone_maneuver_group)
osc_storyboard.add_act(osc_act)
osc_file = xosc.Scenario(
		name=SCENARIO_NAME,
		author="AstaZero 1.0",
		parameters=osc_params,
		entities=osc_entities,
		storyboard=osc_storyboard,
		roadnetwork=osc_road_network,
		catalog=osc_catalogs
	)

osc_file.write_xml(SCENARIO_FILE_NAME)